In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.metrics as metrics
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import itertools
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

In [3]:
ganMode = "DC"
genEpochs = 50

In [4]:
def convNeuralNet(train_images, train_labels, test_images, test_labels):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(96, (3, 3), activation='relu'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    images = np.expand_dims(train_images, axis = -1)
    test_images = np.expand_dims(test_images, axis = -1)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(num_classes = 1)])

    hisotry = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
    
    test_loss, test_acc, test_prec, test_rec, test_fscore = model.evaluate(test_images, test_labels, verbose=2)
    return (test_acc, test_prec, test_rec, test_fscore)

In [ ]:
avg_acc = 0
count = 0
low_acc = 0
high_acc = 0
avg_prec = 0
low_prec = 0
high_prec = 0
avg_rec = 0
low_rec = 0
high_rec = 0
avg_f1s = 0
low_f1s = 0
high_f1s = 0
for comb in itertools.combinations([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 8):
    
    ## Clears lists before each loading
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    
    # Loading of real images for each trial
    for val in range(1, 11):
        if comb.count(val) > 0:
            for i in range((150*(val-1)), (150*val)):
                image = cv2.imread(os.path.join("../cnn-gan-data/Br35H/nontumorous/","no{i}.jpg".format(i=i)))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (128, 128))
                train_images.append(image)
                train_labels.append(0)
                image = cv2.imread(os.path.join("../cnn-gan-data/Br35H/tumorous/","y{i}.jpg".format(i=i)))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (128, 128))
                train_images.append(image)
                train_labels.append(1)
        else:
            for i in range((150*(val-1)), (150*val)):
                image = cv2.imread(os.path.join("../cnn-gan-data/Br35H/nontumorous/","no{i}.jpg".format(i=i)))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (128, 128))
                test_images.append(image)
                test_labels.append(0)
                image = cv2.imread(os.path.join("../cnn-gan-data/Br35H/tumorous/","y{i}.jpg".format(i=i)))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (128, 128))
                test_images.append(image)
                test_labels.append(1)
          
    # Loading of fake images for each trial
    if ganMode != 'None':
        if ganMode == 'DC':
            tumorPath = '../cnn-gan-data/train-images/yes-tumor/dcgan-images'
            noTumorPath = '../cnn-gan-data/train-images/no-tumor/dcgan-images'
        if ganMode == 'Unrolled':
            tumorPath = '../cnn-gan-data/train-images/yes-tumor/unrolled-images'
            noTumorPath = '../cnn-gan-data/train-images/no-tumor/unrolled-images'
        if ganMode == 'W':
            tumorPath = '../cnn-gan-data/train-images/yes-tumor/wgan-images'
            noTumorPath = '../cnn-gan-data/train-images/no-tumor/wgan-images'

        if genEpochs == 5:
            tumorPath = tumorPath + '/Epochs5'
            noTumorPath = noTumorPath + '/Epochs5'
        if genEpochs == 15:
            tumorPath = tumorPath + '/Epochs15'
            noTumorPath = noTumorPath + '/Epochs15'
        if genEpochs == 25:
            tumorPath = tumorPath + '/Epochs25'
            noTumorPath = noTumorPath +'/Epochs25'
        if genEpochs == 50:
            tumorPath = tumorPath + '/Epochs50'
            noTumorPath = noTumorPath +'/Epochs50'
            
        
        for image_name in os.listdir(noTumorPath):
            img_dir = os.path.join(noTumorPath, image_name)
            image = cv2.imread(img_dir)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            train_images.append(image)
            train_labels.append(0)
        
        for image_name in os.listdir(tumorPath):
            img_dir = os.path.join(tumorPath, image_name)
            image = cv2.imread(img_dir)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (128, 128))
            train_images.append(image)
            train_labels.append(1)
    
    # Converting lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    # Increments trial count, runs trial, and outputs results
    count += 1
    metrics = convNeuralNet(train_images, train_labels, test_images, test_labels)
    if low_acc == 0 or low_acc > metrics[0]:
        low_acc = metrics[0]
    if high_acc < metrics[0]:
        high_acc = metrics[0]
    print("Trial {n} accuracy: {acc}".format(n=count, acc=metrics[0]))
    avg_acc += metrics[0]
    
    if low_prec == 0 or low_prec > metrics[1]:
        low_prec = metrics[1]
    if high_prec < metrics[1]:
        high_prec = metrics[1]
    print("Trial {n} precision: {prec}".format(n=count, prec=metrics[1]))
    avg_prec += metrics[1]
    
    if low_rec == 0 or low_rec > metrics[2]:
        low_rec = metrics[2]
    if high_rec < metrics[2]:
        high_rec = metrics[2]
    print("Trial {n} recall: {rec}".format(n=count, rec=metrics[2]))
    avg_rec += metrics[2]
    
    if low_f1s == 0 or low_f1s > metrics[3]:
        low_f1s = metrics[3]
    if high_f1s < metrics[3]:
        high_f1s = metrics[3]
    print("Trial {n} F1Score: {f1s}".format(n=count, f1s=metrics[3]))
    avg_f1s += metrics[3]

avg_acc = avg_acc/count
print("Average accuracy of all trials: {avg_acc}".format(avg_acc=avg_acc))
print("Lowest accuracy of all trials: {low_acc}".format(low_acc=low_acc))
print("Highest accuracy of all trials: {high_acc}".format(high_acc=high_acc))

avg_prec = avg_prec/count
print("Average precision of all trials: {avg_prec}".format(avg_prec=avg_prec))
print("Lowest precision of all trials: {low_prec}".format(low_prec=low_prec))
print("Highest precision of all trials: {high_prec}".format(high_prec=high_prec))

avg_rec = avg_rec/count
print("Average recall of all trials: {avg_rec}".format(avg_rec=avg_rec))
print("Lowest recall of all trials: {low_rec}".format(low_rec=low_rec))
print("Highest recall of all trials: {high_rec}".format(high_rec=high_rec))

avg_f1s = avg_f1s/count
print("Average FScore of all trials: {avg_f1s}".format(avg_f1s=avg_f1s))
print("Lowest FScore of all trials: {low_f1s}".format(low_f1s=low_f1s))
print("Highest FScore of all trials: {high_f1s}".format(high_f1s=high_f1s))